In [37]:
from collections import defaultdict
import json

import numpy as np
import pandas as pd

import time

In [38]:
rfpl_info_file = './Data/rfpl_info.json'
with open(rfpl_info_file, 'r') as file:
    data = json.load(file)

* Get the venues info:

In [39]:
matches = data["data"]["stat_tournament"]["currentSeason"]["matches"]
VENUE_COLS = ['id', 'name', 'capacity', 'lon', 'lat']

In [40]:
matches_info = defaultdict(list)
for match in matches:
    matches_info['match_id'].append(match['id'])
    matches_info['match_start_time'].append(match['scheduledAtStamp'])
    matches_info['home_team_id'].append(match['home']['team']['id'])
    matches_info['home_team_name'].append(match['home']['team']['name'])
    matches_info['home_team_logo'].append(match['home']['team']['logo']['main'])
    matches_info['away_team_id'].append(match['away']['team']['id'])
    matches_info['away_team_name'].append(match['away']['team']['name'])
    matches_info['away_team_logo'].append(match['away']['team']['logo']['main'])
    home_score = 0
    away_score = 0
    for scores in match['periodScore']:
        home_score += scores['homeScore']
        away_score += scores['awayScore']
    matches_info['home_score'].append(home_score)
    matches_info['away_score'].append(away_score)
    if match['venue'] is not None:
        for col in VENUE_COLS:
            matches_info['venue_'+col].append(match['venue'][col])
    else:
        for col in VENUE_COLS:
            matches_info['venue_'+col].append(None)

In [41]:
MATCHES_INFO_PATH = './Data/matches_info.csv'
matches_df = pd.DataFrame(
    data=matches_info
)
matches_df.dropna(inplace=True)
matches_df.to_csv(MATCHES_INFO_PATH)
matches_df

,match_id,match_start_time,home_team_id,home_team_name,home_team_logo,away_team_id,away_team_name,away_team_logo,home_score,away_score,venue_id,venue_name,venue_capacity,venue_lon,venue_lat
0,000000000001,1562950800,fc_lokomotiv_moscow,Локомотив,https://statistics-stat-logos.trbna.com/champi...,zenit,Зенит,https://statistics-stat-logos.trbna.com/europe...,1,0,rzd_arena,РЖД Арена,28800.0,37.560816,55.659005
1,18382061,1562950800,arsenal_tula,Арсенал Тула,https://s5o.ru/storage/simple/ru/edt/19/34/88/...,fc_dinamo_moscow,Динамо,https://s5o.ru/storage/simple/ru/edt/16/ab/10/...,1,1,arsenal_stadium,Арсенал,20074.0,37.603056,54.174167
2,18382065,1563024600,fc_ural_yekaterinburg,Урал Екатеринбург,https://s5o.ru/storage/simple/ru/edt/11/49/70/...,fc_ufa,Уфа,https://s5o.ru/storage/simple/ru/edt/85/49/70/...,3,2,ekaterinburg_arena,Central Stadium,35061.0,60.573611,56.832500
3,18382053,1563033600,spartak_moscow,Спартак,https://s5o.ru/storage/simple/ru/edt/20/36/94/...,dinamo_st_pburg,Сочи,https://s5o.ru/storage/simple/ru/edt/45/f3/da/...,1,0,otkritie_arena,Открытие Арена,44360.0,37.440250,55.817861
4,18382055,1563042600,fc_rostov,Ростов,https://s5o.ru/storage/simple/ru/edt/06/36/53/...,fc_orenburg,Оренбург,https://s5o.ru/storage/simple/ru/edt/27/22/66/...,2,1,rostov_arena,Ростов Арена,45000.0,39.737778,47.209444
5,18382063,1563111000,fc_kryliya_sovetov_samara,Крылья Советов,https://s5o.ru/storage/simple/ru/edt/64/22/33/...,cska_moscow,ЦСКА,https://statistics-stat-logos.trbna.com/champi...,2,0,samara_arena,Самара Арена,44918.0,50.237222,53.277778
6,18382051,1563120000,zenit,Зенит,https://statistics-stat-logos.trbna.com/europe...,fk_tambov,ФК Тамбов,https://s5o.ru/storage/simple/ru/edt/bb/cb/49/...,2,1,saint_petersburg_stadium,Стадион Санкт-Петербург,70000.0,30.220533,59.972953
7,18382059,1563129000,fc_akhmat_grozny,Ахмат,https://s5o.ru/storage/simple/ru/edt/eb/e9/7c/...,fc_krasnodar,Краснодар,https://statistics-stat-logos.trbna.com/europe...,1,0,akhmat-arena,Ахмат-Арена,30597.0,45.745252,43.323176
8,18382057,1563210000,fc_lokomotiv_moscow,Локомотив,https://statistics-stat-logos.trbna.com/champi...,fc_rubin_kazan,Рубин Казань,https://s5o.ru/storage/simple/ru/edt/10/85/3e/...,1,1,rzd_arena,РЖД Арена,28800.0,37.741111,55.803611
9,18382079,1563620400,fc_ufa,Уфа,https://s5o.ru/storage/simple/ru/edt/85/49/70/...,fc_krasnodar,Краснодар,https://statistics-stat-logos.trbna.com/europe...,2,3,neftyanik_stadium,Нефтяник,15234.0,56.061450,54.825864


* Match the User's location and time with the suitable match:

In [42]:
def return_closest_match_info(input_dict, matches_db_path='./Data/matches_info.csv'):
    '''
    input_dict = {'lon': 0, 'lat': 0, 'time': unicode_time}
    '''
    matches_df = pd.read_csv(matches_db_path)
    lonlat = matches_df.loc[:, ['venue_lon', 'venue_lat']].values
    user_lonlat = np.array([input_dict['lon'], input_dict['lat']])
    dists = np.square(lonlat[:,np.newaxis]-user_lonlat).sum(axis=2)
    closest_venue_id = matches_df.iloc[np.argmin(dists),:]['venue_id']
    closest_matches = matches_df[matches_df['venue_id'] == closest_venue_id]
    closest_match_id = np.argmin(closest_matches['match_start_time'].values - input_dict['time'])
    result = dict(closest_matches.iloc[closest_match_id, :])
    INT_COLS = ['match_id', 'match_start_time', 'venue_capacity']
    FLOAT_COLS = ['venue_lon', 'venue_lat']
    for col in INT_COLS:
        result[col] = int(result[col])
    for col in FLOAT_COLS:
        result[col] = float(result[col])
    del result['Unnamed: 0']
    return result

In [43]:
input_dict = {'lon': 32.0547, 'lat': 49.4361, 'time': 1564419600}                                              
out = return_closest_match_info(input_dict)

In [44]:
out

{'match_id': 18382091,
 'match_start_time': 1564419600,
 'home_team_id': 'fc_rubin_kazan',
 'home_team_name': 'Рубин Казань',
 'home_team_logo': 'https://s5o.ru/storage/simple/ru/edt/10/85/3e/55/rueb32ba662b6.jpg',
 'away_team_id': 'fc_akhmat_grozny',
 'away_team_name': 'Ахмат',
 'away_team_logo': 'https://s5o.ru/storage/simple/ru/edt/eb/e9/7c/35/ruee5d9c2d146.jpg',
 'home_score': 1,
 'away_score': 0,
 'venue_id': 'tsentralnyi_stadium',
 'venue_name': 'Центральный стадион',
 'venue_capacity': 10321,
 'venue_lon': 32.0547,
 'venue_lat': 49.4361}

In [36]:
for col in out:
    print(type(out[col]))

<class 'int'>
<class 'int'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'int'>
<class 'float'>
<class 'float'>
